In [7]:
import pandas as pd
import re
import duckdb
import time
print(f"Required libraries imported successfully")


Required libraries imported successfully


In [8]:
def cleanColumnHeader(df):
    print(f"Old column name {df.columns}")
    df.columns = [re.sub(" ", "_", re.sub(" {1,}", " ", re.sub("/", " ", str(col).lower()))) for col in df.columns]
    print(f"New column name {df.columns}")

## Reading the csv file

In [9]:
df = pd.read_csv(r"./data/Air_Traffic_Passenger_Statistics.csv")
df.head(2)


,Activity Period,Operating Airline,Operating Airline IATA Code,Published Airline,Published Airline IATA Code,GEO Summary,GEO Region,Activity Type Code,Price Category Code,Terminal,Boarding Area,Passenger Count
0,200507,ATA Airlines,TZ,ATA Airlines,TZ,Domestic,US,Deplaned,Low Fare,Terminal 1,B,27271
1,200507,ATA Airlines,TZ,ATA Airlines,TZ,Domestic,US,Enplaned,Low Fare,Terminal 1,B,29131


## Data Cleaning and Transformations

In [10]:
if df.shape[0] > 0:
    print(f"File successfully imported")
    cleanColumnHeader(df)

File successfully imported
Old column name Index(['Activity Period', 'Operating Airline', 'Operating Airline IATA Code',
       'Published Airline', 'Published Airline IATA Code', 'GEO Summary',
       'GEO Region', 'Activity Type Code', 'Price Category Code', 'Terminal',
       'Boarding Area', 'Passenger Count'],
      dtype='object')
New column name Index(['activity_period', 'operating_airline', 'operating_airline_iata_code',
       'published_airline', 'published_airline_iata_code', 'geo_summary',
       'geo_region', 'activity_type_code', 'price_category_code', 'terminal',
       'boarding_area', 'passenger_count'],
      dtype='object')


In [11]:
df = df.assign(
    year_month=pd.to_datetime(df["activity_period"], format="%Y%m").apply(
        lambda x: x.strftime("%B-%Y")
    ),
    year=pd.to_datetime(df["activity_period"], format="%Y%m").dt.strftime("%Y"),
)

df["operating_airline"] = df["operating_airline"].str.replace(
    "United Airlines - Pre 07/01/2013", "United Airlines"
)

## Total Passenger Count of every Airlines

In [12]:
# we will aggregate the data on Airlines and calculate total_passenger_count as sum of over passenger count for every airlines

df_agg_airline_count = (
                    df.groupby(['operating_airline']) # aggregating data for every airline
                    .agg(total_passenger_count=("passenger_count","sum")) # adding the passenger count from 2005-2022(entire dataset)
                    .reset_index() # make operating_airline as column again
                    .sort_values(by = ['total_passenger_count'], ascending=[False])
                    )
#df_summary_overall.reset_index(inplace = True)
df_agg_airline_count.head()




,operating_airline,total_passenger_count
91,United Airlines,560898827
79,SkyWest Airlines,118974403
18,American Airlines,110452149
35,Delta Air Lines,94324906
80,Southwest Airlines,84391174


## Top 10 Airlines by Total Passenger Count

In [13]:
# to get the N top rows we can make use of nlargest function after aggregating the data and supply 
# the column based on which largest should be decided and value of N
N = 10 # Define the value of N to find the top N rows

df_top_N_airlines = (
                    df.groupby(['operating_airline']) # aggregating data for every airline
                    .agg(total_passenger_count=("passenger_count","sum")) # adding the passenger count from 2005-2022(entire dataset)
                    .reset_index()
                    .nlargest(N, 'total_passenger_count') # we pass N i.e. 10 and column on which largest will be decided
                    )

df_top_N_airlines

,operating_airline,total_passenger_count
91,United Airlines,560898827
79,SkyWest Airlines,118974403
18,American Airlines,110452149
35,Delta Air Lines,94324906
80,Southwest Airlines,84391174
93,Virgin America,75687840
15,Alaska Airlines,64400258
90,US Airways,33633232
56,JetBlue Airways,32415972
5,Air Canada,22316453


## Top N Airlines every year since 2005 by Passenger Count

In [14]:
# Now requirement is to get top N airlines of every year by Passenger count. So now we will use nth function of groupby
# This function will give nth rows of every group

In [15]:
N = 5 # Define the top N 
df_summary = (
    df[["year", "operating_airline","passenger_count" ]]
    .groupby(["year", "operating_airline"])
    .agg(total_passenger_count=("passenger_count", "sum"))
    .reset_index()
    .sort_values(["year", "total_passenger_count"], ascending=[True, False])
    .groupby(["year"], as_index=False, sort=False)
    .nth[:N] # nth function requires min pandas 1.4.0
)
df_summary[df_summary.year == '2021']

,year,operating_airline,total_passenger_count
813,2021,United Airlines,18625388
805,2021,SkyWest Airlines,6773032
780,2021,American Airlines,4652840
786,2021,Delta Air Lines,4273302
778,2021,Alaska Airlines,3156246


## Creating Animation using Ipyvizzu

We will create Horizontal Bar plot showing passenger count of top 5 airlines every year

In [16]:
from ipyvizzu import Chart, Data, Config, Style, DisplayTarget

In [17]:
years = df_summary["year"].sort_values().unique()
data = Data()
data.add_data_frame(df_summary)  # add dataframe to ipyvizzu
chart = Chart(width="100%", display=DisplayTarget.MANUAL)
config = {
    "channels": {
        "y": {
            "set": ["operating_airline"],
        },
        "x": {
            "set": ["total_passenger_count"],
            "labels": True,
            #"range": {"min": "20%", "max": "120%"},
            # Changed range to fixed value to show year on year change as suggested by Peter
            "range": {"min": 100000, "max": 50000000}, 
        },
        "label": {"set": ["total_passenger_count"]},
        "color": {"set": ["operating_airline"]},
    },
    "sort": "byValue",
}

style = Style(
    {
        "plot": {
            "paddingLeft": 150,
            "paddingTop": 25,
            "paddingRight": 20,
            "yAxis": {
                "color": "#ffffff00",
                "label": {"paddingRight": 10},
            },
            "xAxis": {
                "title": {"color": "#ffffff00"},
                "label": {
                    "color": "#ffffff00",
                    # "numberFormat": "grouped",
                    # "numberFormat": "prefixed",
                    # "maxFractionDigits":"0",
                },
            },
            "marker": {
                "colorPalette": "#d9ed92ff #b5e48cff #99d98cff #76c893ff #52b69aff"
                + " #34a0a4ff #168aadff #1a759fff #1e6091ff #184e77ff",
                "label": {
                    "maxFractionDigits": 2,
                    "numberFormat": "prefixed",
                    "fontWeight": "bold",
                    "fontSize": 12,
                    "position": "right",
                },  # to remove decimal digits from marker label
            },
        },
    }
)

#chart = Chart(display=DisplayTarget.END)


# chart.animate(data.filter(filter_format), style)
chart.animate(data, style)
# chart.animate(Config(config))

for year in years:
    config["title"] = f"Airlines Passenger Count in {year}"
    # print(month)
    chart.animate(
        Data.filter(f"parseInt(record.year) == {year}"),
        Config(config),
        duration=1,
        x={"easing": "linear", "delay": 0},
        y={"delay": 0},
        show={"delay": 0},
        hide={"delay": 0},
        title={"duration": 0, "delay": 0},
    )

chart.animate(
    Config(
        {
            "channels": {
                "x": {"attach": ["year"],},
                # "label": {"set": "Passenger Count"},
            }
        },
    ),
    
    Style({"plot": {"marker": {"label": {"position": "right"}}}}),
    duration=1,
)
chart.show()

## Now getting the same output using DuckDB and SQL

We will write a sql query which will be executed using DUCKDB engine

In [18]:
n = 10 # Defining top N
# this query will give top n rows of each year based on passenger count.
sql_query = f""" 
    SELECT
    final.Year as Year,
    cast(final.Total_Passenger_Count as int) as Total_Passenger_Count,
    final.operating_airline as Airlines,
    final.rn
from
    (
        /* This inner query using row_number will partition result based on year and give row number based on passenger count */
        SELECT
            a.*,
            row_number() OVER (
                PARTITION by a.year
                order by
                    a.year,
                    a.Total_Passenger_Count desc
            ) as rn
        FROM
            (
                /* This inner query will aggregate the data based on year and airline and get the passenger count*/
                SELECT
                    SUM("passenger_count") as "Total_Passenger_Count",
                    "year",
                    "operating_airline"
                FROM
                    df
                group by
                    "year",
                    "operating_airline"
            ) a
    ) final
where
    rn <= {n} order by Year, Total_Passenger_Count desc"""

In [19]:

df_summary_duckdb = duckdb.query(sql_query).to_df()

df_summary_duckdb.head(10)

,year,Total_Passenger_Count,Airlines,rn
0,2005,15338298,United Airlines,1
1,2005,3438824,American Airlines,2
2,2005,2568754,SkyWest Airlines,3
3,2005,2072072,Delta Air Lines,4
4,2005,1731352,US Airways,5
5,2005,1384590,Northwest Airlines (became Delta),6
6,2005,1285300,Alaska Airlines,7
7,2005,617492,Air Canada,8
8,2005,555080,ATA Airlines,9
9,2005,457400,Lufthansa German Airlines,10


In [20]:
df_summary_duckdb.head(20)

,year,Total_Passenger_Count,Airlines,rn
0,2005,15338298,United Airlines,1
1,2005,3438824,American Airlines,2
2,2005,2568754,SkyWest Airlines,3
3,2005,2072072,Delta Air Lines,4
4,2005,1731352,US Airways,5
5,2005,1384590,Northwest Airlines (became Delta),6
6,2005,1285300,Alaska Airlines,7
7,2005,617492,Air Canada,8
8,2005,555080,ATA Airlines,9
9,2005,457400,Lufthansa German Airlines,10


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=76e2df7a-4a6e-4112-9680-1feec18bdf23' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>